## Classification

To run this script, you need the following files found in the /data directory:
- "final_labels_SG1.xlsx"
- "final_labels_SG2.xlsx"

## Imports and set-up

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 9.1MB/s 
     |████████████████████████████████| 3.3MB 33.7MB/s 
     |████████████████████████████████| 901kB 46.9MB/s 
     |████████████████████████████████| 1.2MB 8.0MB/s 


In [2]:
!nvidia-smi

Wed May 12 08:22:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import sys
import os
import time
import re
import random
from typing import Dict, List, Optional, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from transformers import BertTokenizer, BertConfig, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import LongformerTokenizer, TFLongformerForSequenceClassification

Mounted at /content/drive
/content


In [4]:
# set seed, TF uses python ramdom and numpy library, so these must also be fixed
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
os.environ['TF_DETERMINISTIC_OPS'] = '0'

In [5]:
# see if hardware accelerator available
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

If GPUs are available, tensorflow will give priority to it automatically and computations will be performed on the GPU as default. That behavior can be changed by assigning a task explicitly to a device. Example:

```
with tf.device('/CPU:0'):
```



## Preprocessing

In [28]:
PATH_sg1 = "data/final_labels_SG1.xlsx"
PATH_sg2 = "data/final_labels_SG2.xlsx"
df_sg1 = pd.read_excel(PATH_sg1)
df_sg2 = pd.read_excel(PATH_sg2)
df_sg1.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg2.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg1.head()

,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,Non-biased,Entirely factual,[]
1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,No agreement,Entirely factual,[]
2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,Biased,Entirely factual,[]
3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,Biased,Entirely factual,[]
4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,Biased,Entirely factual,[]


In [29]:
# binarize classification problem
df_sg1 = df_sg1[df_sg1['Label_bias']!='No agreement']
df_sg1 = df_sg1[df_sg1['Label_bias'].isna()==False]
df_sg1.replace(to_replace='Biased', value=1, inplace=True)
df_sg1.replace(to_replace='Non-biased', value=0, inplace=True)

df_sg2 = df_sg2[df_sg2['Label_bias']!='No agreement']
df_sg2.replace(to_replace='Biased', value=1, inplace=True)
df_sg2.replace(to_replace='Non-biased', value=0, inplace=True)

In [23]:
# Stratified k-Fold instance
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

The rest of the preprocessing needs to be performed inside the folds as a) encoder layers shouldn't be allowed to see whole data to construct the lookups and b) indexing with skfold is not possible when data is in tensorflow format.

In [24]:
# helper functions called in skfold loop

def pd_to_tf(df):
    """convert a pandas dataframe into a tensorflow dataset"""
    target = df.pop('Label_bias')
    sentence = df.pop('sentence')
    return tf.data.Dataset.from_tensor_slices((sentence.values, target.values))

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

def tokenize(df):
    """convert a pandas dataframe into a tensorflow dataset and run hugging face's tokenizer on data"""
    target = df.pop('Label_bias')
    sentence = df.pop('sentence')

    train_encodings = tokenizer(
                        sentence.tolist(),                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        truncation = True, # cut off at max length of the text that can go to BERT
                        padding = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(train_encodings), 
         target.tolist()))
    return dataset

## Attention-based models


In [25]:
def run_model_5fold(df_train, model_name, freeze_encoder=True, pretrained=False, plot=False):
  """"freeze flags whether encoder layer should be frozen to not destroy transfer learning. Only set to false when enough data is provided"""

  # these variables will be needed for skfold to select indices
  Y = df_train['Label_bias']
  X = df_train['sentence']

  # hyperparams
  BUFFER_SIZE = 10000
  BATCH_SIZE = 32
  k = 1

  val_loss = []
  val_acc = []
  val_prec = []
  val_rec = []
  val_f1 = []
  val_f1_micro = []
  val_f1_wmacro = []

  for train_index, val_index in skfold.split(X,Y):
    print('### Start fold {}'.format(k))
    
    # split into train and validation set
    train_dataset = df_train.iloc[train_index]
    val_dataset = df_train.iloc[val_index]

    # prepare data for transformer
    train_dataset = tokenize(train_dataset)
    val_dataset = tokenize(val_dataset)

    # mini-batch it
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

    # create new model
    if model_name == 'bert':
      model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
    if model_name == 'distilbert':
      model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
    elif model_name == 'roberta':
      model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
    elif model_name == 'electra':
      model = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
    elif model_name == 'xlnet':
      model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased')


    if freeze_encoder == True:
      for w in model.get_layer(index=0).weights:
        w._trainable = False

    # compile it
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
    model.compile(optimizer=optimizer, loss=model.compute_loss) 

    # transfer learning
    if pretrained == True:
      model.get_layer(index=0).set_weights(trained_model_layer) # load bias-specific weights
      #model.load_weights('./checkpoints/')
    
    # after 2 epochs without improvement, stop training
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

    # fit it
    history = model.fit(train_dataset, epochs=10, validation_data = val_dataset, callbacks=[callback])
    
    # plot history
    if plot:
      plot_graphs(history,'loss')

    # evaluate
    loss = model.evaluate(val_dataset)
    
    if model_name == 'xlnet':
      yhats = []
      for row in df_train.iloc[val_index]['sentence']:
        input = tokenizer(row, return_tensors="tf")
        output = model(input)
        logits = output.logits.numpy()[0]
        candidates = logits.tolist()
        decision = candidates.index(max(candidates))
        yhats.append(decision)
    else:
      logits = model.predict(val_dataset)  
      yhats = []
      for i in logits[0]:
        # assign class label according to highest logit
        candidates = i.tolist()
        decision = candidates.index(max(candidates))
        yhats.append(decision)
    
    y = []
    for text, label in val_dataset.unbatch():   
      y.append(label.numpy())
    
    val_loss.append(loss)
    val_acc.append(accuracy_score(y, yhats))
    val_prec.append(precision_score(y, yhats))
    val_rec.append(recall_score(y, yhats))
    val_f1.append(f1_score(y, yhats))
    val_f1_micro.append(f1_score(y, yhats, average='micro'))
    val_f1_wmacro.append(f1_score(y, yhats, average='weighted'))

    tf.keras.backend.clear_session()

    k += 1

  return val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro

### BERT

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# without distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=False)

# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autog

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 47s 850ms/step - loss: 0.6913 - val_loss: 0.6510
Epoch 2/10
39/39 [==============================] - 31s 802ms/step - loss: 0.4693 - val_loss: 0.5219
Epoch 3/10
10/10 [==============================] - 2s 193ms/step - loss: 0.5219
### Start fold 3


...

Epoch 1/10
39/39 [==============================] - 49s 872ms/step - loss: 0.6901 - val_loss: 0.5286
Epoch 2/10
39/39 [==============================] - 32s 812ms/step - loss: 0.4648 - val_loss: 0.4567
Epoch 3/10
10/10 [==============================] - 3s 250ms/step - loss: 0.4567
### Start fold 4


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 48s 854ms/step - loss: 0.7011 - val_loss: 0.5454
Epoch 2/10
39/39 [==============================] - 31s 796ms/step - loss: 0.4544 - val_loss: 0.5303
Epoch 3/10
10/10 [==============================] - 2s 193ms/step - loss: 0.5303
### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 49s 882ms/step - loss: 0.6654 - val_loss: 0.5702
Epoch 2/10
39/39 [==============================] - 32s 818ms/step - loss: 0.4583 - val_loss: 0.4501
Epoch 3/10
10/10 [==============================] - 3s 251ms/step - loss: 0.4501
5-Fold CV Loss: 0.5055139303207398
5-Fold CV Accuracy: 0.7649161518093557
5-Fold CV Precision: 0.7723268878256538
5-Fold CV Recall: 0.7543624161073825
5-Fold CV F1 Score: 0.7516630925882166
5-Fold CV Micro F1 Score: 0.7649161518093556
5-Fold CV Weighted Macro F1 Score: 0.7618697657898658


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# without distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=False)

In [34]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT on SG2
5-Fold CV Loss: 0.4575493991374969
5-Fold CV Accuracy: 0.7892706074255316
5-Fold CV Precision: 0.7796414062302606
5-Fold CV Recall: 0.8049723756906078
5-Fold CV F1 Score: 0.7900329021285127
5-Fold CV Micro F1 Score: 0.7892706074255315
5-Fold CV Weighted Macro F1 Score: 0.7886159511454031


### BERT + distant

In [37]:
# load model layer weights from pretraining on distant dataset 
# compile model
#transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
transfer_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(optimizer=optimizer, loss=transfer_model.compute_loss) 

transfer_model.load_weights('./checkpoints/bert_final_checkpoint_news_headlines_USA')
trained_model_layer = transfer_model.get_layer(index=0).get_weights()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# with distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=True)

In [39]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT + distant on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT + distant on SG1
5-Fold CV Loss: 0.4784796714782715
5-Fold CV Accuracy: 0.7791283150506451
5-Fold CV Precision: 0.782222153066104
5-Fold CV Recall: 0.7624161073825504
5-Fold CV F1 Score: 0.7690257043162897
5-Fold CV Micro F1 Score: 0.7791283150506451
5-Fold CV Weighted Macro F1 Score: 0.7782424758051683


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# with distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=True)

In [41]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT + distant on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT + distant on SG2
5-Fold CV Loss: 0.45280778408050537
5-Fold CV Accuracy: 0.8047893380785556
5-Fold CV Precision: 0.8010278449136129
5-Fold CV Recall: 0.8082872928176796
5-Fold CV F1 Score: 0.8027645188619399
5-Fold CV Micro F1 Score: 0.8047893380785558
5-Fold CV Weighted Macro F1 Score: 0.8043577261879854


### DistilBERT

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='distilbert', 
                                                                                            freeze_encoder=False, pretrained=False)

In [36]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for DistilBERT on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for DistilBERT on SG1
5-Fold CV Loss: 0.5034848332405091
5-Fold CV Accuracy: 0.7603517841381919
5-Fold CV Precision: 0.7851353351520555
5-Fold CV Recall: 0.7006711409395974
5-Fold CV F1 Score: 0.736955546776338
5-Fold CV Micro F1 Score: 0.7603517841381919
5-Fold CV Weighted Macro F1 Score: 0.758224422060177


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='distilbert', 
                                                                                            freeze_encoder=False, pretrained=False)

In [43]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for DistilBERT on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for DistilBERT on SG2
5-Fold CV Loss: 0.47459145188331603
5-Fold CV Accuracy: 0.7783788392741292
5-Fold CV Precision: 0.7914496007137517
5-Fold CV Recall: 0.756353591160221
5-Fold CV F1 Score: 0.768966991077014
5-Fold CV Micro F1 Score: 0.7783788392741293
5-Fold CV Weighted Macro F1 Score: 0.777102422253571


### RoBERTa

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=False)

In [45]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa on SG1
5-Fold CV Loss: 0.4697426736354828
5-Fold CV Accuracy: 0.7784621527339974
5-Fold CV Precision: 0.7825771209819836
5-Fold CV Recall: 0.7758389261744967
5-Fold CV F1 Score: 0.7715627724463581
5-Fold CV Micro F1 Score: 0.7784621527339974
5-Fold CV Weighted Macro F1 Score: 0.7752193988259601


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=False)

In [47]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa on SG2
5-Fold CV Loss: 0.45095362067222594
5-Fold CV Accuracy: 0.8007054811025227
5-Fold CV Precision: 0.7834945430249991
5-Fold CV Recall: 0.8337016574585636
5-Fold CV F1 Score: 0.8042913565026074
5-Fold CV Micro F1 Score: 0.8007054811025227
5-Fold CV Weighted Macro F1 Score: 0.799347878132545


### RoBERTa + distant

In [48]:
# load model layer weights from pretraining on distant dataset 
# compile model
transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(optimizer=optimizer, loss=transfer_model.compute_loss) 

transfer_model.load_weights('./checkpoints/roberta_final_checkpoint_news_headlines_USA')
trained_model_layer = transfer_model.get_layer(index=0).get_weights()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=True)

In [50]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa + distant on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa + distant on SG1
5-Fold CV Loss: 0.44771517515182496
5-Fold CV Accuracy: 0.7985668053629219
5-Fold CV Precision: 0.7758359785184737
5-Fold CV Recall: 0.832214765100671
5-Fold CV F1 Score: 0.7999256262460596
5-Fold CV Micro F1 Score: 0.7985668053629217
5-Fold CV Weighted Macro F1 Score: 0.7977665899984501


In [51]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=True)

### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 97s 877ms/step - loss: 0.5345 - val_loss: 0.4133
Epoch 2/10
23/23 [==============================] - 5s 234ms/step - loss: 0.4133
### Start fold 2


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 95s 874ms/step - loss: 0.5673 - val_loss: 0.4084
Epoch 2/10
92/92 [==============================] - 77s 841ms/step - loss: 0.3456 - val_loss: 0.4032
Epoch 3/10
23/23 [==============================] - 5s 215ms/step - loss: 0.4032
### Start fold 3


...

Epoch 1/10
92/92 [==============================] - 96s 875ms/step - loss: 0.5386 - val_loss: 0.4585
Epoch 2/10
23/23 [==============================] - 5s 222ms/step - loss: 0.4585
### Start fold 4


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 97s 882ms/step - loss: 0.5422 - val_loss: 0.4699
Epoch 2/10
92/92 [==============================] - 78s 845ms/step - loss: 0.3178 - val_loss: 0.4695
Epoch 3/10
23/23 [==============================] - 5s 233ms/step - loss: 0.4695
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 89s 800ms/step - loss: 0.5482 - val_loss: 0.4198
Epoch 2/10
23/23 [==============================] - 6s 263ms/step - loss: 0.4198


In [52]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa + distant on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa + distant on SG2
5-Fold CV Loss: 0.4328740358352661
5-Fold CV Accuracy: 0.8007017738975699
5-Fold CV Precision: 0.8479610779798031
5-Fold CV Recall: 0.72707182320442
5-Fold CV F1 Score: 0.7811997062364262
5-Fold CV Micro F1 Score: 0.8007017738975699
5-Fold CV Weighted Macro F1 Score: 0.7990983671260622


### ELECTRA

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='electra', 
                                                                                            freeze_encoder=False, pretrained=False)

In [54]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for ELECTRA on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for ELECTRA on SG1
5-Fold CV Loss: 0.5440825581550598
5-Fold CV Accuracy: 0.7448009918883706
5-Fold CV Precision: 0.7614052309887166
5-Fold CV Recall: 0.6966442953020134
5-Fold CV F1 Score: 0.7211321040106528
5-Fold CV Micro F1 Score: 0.7448009918883706
5-Fold CV Weighted Macro F1 Score: 0.7422210680339413


In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='electra', 
                                                                                            freeze_encoder=False, pretrained=False)

In [56]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for ELECTRA on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for ELECTRA on SG2
5-Fold CV Loss: 0.5140148580074311
5-Fold CV Accuracy: 0.7625935605849968
5-Fold CV Precision: 0.809470758694404
5-Fold CV Recall: 0.6861878453038674
5-Fold CV F1 Score: 0.7379172614803242
5-Fold CV Micro F1 Score: 0.7625935605849969
5-Fold CV Weighted Macro F1 Score: 0.7597981280791523


### XLNET

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='xlnet', 
                                                                                            freeze_encoder=False, pretrained=False)

In [58]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for XLNET on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for XLNET on SG1
5-Fold CV Loss: 0.5028677642345428
5-Fold CV Accuracy: 0.7616525868953054
5-Fold CV Precision: 0.7908365542157949
5-Fold CV Recall: 0.6939597315436242
5-Fold CV F1 Score: 0.7374808169992624
5-Fold CV Micro F1 Score: 0.7616525868953054
5-Fold CV Weighted Macro F1 Score: 0.7600315195814671


In [59]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='xlnet', 
                                                                                            freeze_encoder=False, pretrained=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Start fold 1


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 120s 1s/step - loss: 0.5646 - val_loss: 0.4400
Epoch 2/10
23/23 [==============================] - 6s 274ms/step - loss: 0.4400
### Start fold 2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/10
92/92 [==============================] - 120s 1s/step - loss: 0.6138 - val_loss: 0.4555
Epoch 2/10
92/92 [==============================] - 102s 1s/step - loss: 0.4001 - val_loss: 0.4433
Epoch 3/10
23/23 [==============================] - 6s 283ms/step - loss: 0.4433
### Start fold 4


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 121s 1s/step - loss: 0.5813 - val_loss: 0.4738
Epoch 2/10
23/23 [==============================] - 7s 324ms/step - loss: 0.4738
### Start fold 5


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 113s 1s/step - loss: 0.5735 - val_loss: 0.4463
Epoch 2/10
23/23 [==============================] - 8s 350ms/step - loss: 0.4463


In [60]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for XLNET on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for XLNET on SG2
5-Fold CV Loss: 0.4420862317085266
5-Fold CV Accuracy: 0.7974357263341304
5-Fold CV Precision: 0.8162492174406484
5-Fold CV Recall: 0.76353591160221
5-Fold CV F1 Score: 0.7870615420108473
5-Fold CV Micro F1 Score: 0.7974357263341304
5-Fold CV Weighted Macro F1 Score: 0.7967053530729449
